## Training the model

In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from unidecode import unidecode
import xgboost as xgb
import boto3
from datetime import datetime
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

In [2]:
data = pd.read_csv("data_totallossco.csv")
data.head(2)

,numero_siniestro,valor_vehiculo,valor_deducible,descripcion_siniestro,1.- Sección delantera,2.- Lateral delantero izquierdo,3.- Lateral delantero derecho,4.- Lateral trasero izquierdo,5.- Lateral trasero derecho,6.- Sección posterior,7.- Techo,codigo_marca,codigo_clase,codigo_tipo_vehiculo,anio_modelo,municipio,label
0,899767,68900000.0,10% con un mínímo de 6 SMMLV,"El día primero (1) de enero de 2022, siendo ap...",X,X,X,X,X,X,X,16,12,138,2012,GRANADA,0
1,899768,73700000.0,$950.000,El vehículo venía de Istmina Chocó a recogerm...,X,X,X,X,X,X,X,30,6,134,2020,PEREIRA,1


We will standarize the values of valor_deducible since there are different formats

In [3]:
def normalize_text(value): 
    if pd.isna(value):
        return None

    # Lowercase, remove extra spaces, and standardize similar terms
    value = value.lower().strip()
    value = value.replace('í', 'i').replace(',', '.').replace('%', '').replace('$', '')
    value = value.strip()
    value = re.sub(r'\s+', ' ', value)  # Replace multiple spaces with a single space

    return value

# Apply the normalization function to the 'valor_deducible' column
data['valor_deducible_normalized'] = data['valor_deducible'].apply(normalize_text)

# Display unique normalized values for verification
normalized_unique_values = data['valor_deducible_normalized'].unique()
normalized_unique_values

array(['10 con un minimo de 6 smmlv', '950.000',
       '10 con un minimo de 1.5 smmlv', '10 con un minimo de 1 smmlv',
       '0 con un minimo de 1 smmlv', '0 con un minimo de 1.5 smmlv',
       '1.300.000', '2.000.000', '3.000.000',
       '10 con un minimo de 3 smmlv', 'sin deducible',
       '10 con un minimo de 2 smmlv', '4.400.000',
       '20 con un minimo de 4 smmlv', '20 con un minimo de 1.5 smmlv',
       '15 con un minimo de 10 smmlv', '0 con un minimo de 1.2 smmlv',
       '4.000.000', '1.600.000', '20 con un minimo de 1 smmlv',
       '20 con un minimo de 6 smmlv', '10 con un minimo de 4 smmlv',
       '30 con un minimo de 2 smmlv', '20 con un minimo de 3 smmlv',
       '10 con un minimo de 0.5 smmlv', None,
       '15 con un minimo de 3 smmlv', '20 con un minimo de 0 smmlv',
       '1.400.000', '0 con un minimo de 0 smmlv',
       '15 con un minimo de 6 smmlv', '2.500.000', '0',
       '20 con un minimo de 2 smmlv', '10 con un minimo de 0 smmlv',
       '15 con un minimo 

In [4]:
data.columns

Index(['numero_siniestro', 'valor_vehiculo', 'valor_deducible',
       'descripcion_siniestro', '1.- Sección delantera',
       '2.- Lateral delantero izquierdo', '3.- Lateral delantero derecho',
       '4.- Lateral trasero izquierdo', '5.- Lateral trasero derecho',
       '6.- Sección posterior', '7.- Techo', 'codigo_marca', 'codigo_clase',
       'codigo_tipo_vehiculo', 'anio_modelo', 'municipio', 'label',
       'valor_deducible_normalized'],
      dtype='object')

In [5]:
columns_to_encoder = ['1.- Sección delantera',
       '2.- Lateral delantero izquierdo', '3.- Lateral delantero derecho',
       '4.- Lateral trasero izquierdo', '5.- Lateral trasero derecho',
       '6.- Sección posterior', '7.- Techo']

In [6]:
for c in columns_to_encoder:
    data[c] = data[c].apply(lambda x: 1 if x == "X" else 0)

In [7]:
data['num_partes_afectadas'] = data[columns_to_encoder].sum(axis=1)

In [8]:
data.head(2)

,numero_siniestro,valor_vehiculo,valor_deducible,descripcion_siniestro,1.- Sección delantera,2.- Lateral delantero izquierdo,3.- Lateral delantero derecho,4.- Lateral trasero izquierdo,5.- Lateral trasero derecho,6.- Sección posterior,7.- Techo,codigo_marca,codigo_clase,codigo_tipo_vehiculo,anio_modelo,municipio,label,valor_deducible_normalized,num_partes_afectadas
0,899767,68900000.0,10% con un mínímo de 6 SMMLV,"El día primero (1) de enero de 2022, siendo ap...",1,1,1,1,1,1,1,16,12,138,2012,GRANADA,0,10 con un minimo de 6 smmlv,7
1,899768,73700000.0,$950.000,El vehículo venía de Istmina Chocó a recogerm...,1,1,1,1,1,1,1,30,6,134,2020,PEREIRA,1,950.000,7


In [9]:
data = data.drop(columns = ['valor_deducible'])

In [10]:
data.rename(columns = {'valor_deducible_normalized': 'valor_deducible'}, inplace = True)

In [11]:
data.dtypes

numero_siniestro                     int64
valor_vehiculo                     float64
descripcion_siniestro               object
1.- Sección delantera                int64
2.- Lateral delantero izquierdo      int64
3.- Lateral delantero derecho        int64
4.- Lateral trasero izquierdo        int64
5.- Lateral trasero derecho          int64
6.- Sección posterior                int64
7.- Techo                            int64
codigo_marca                         int64
codigo_clase                         int64
codigo_tipo_vehiculo                 int64
anio_modelo                          int64
municipio                           object
label                                int64
valor_deducible                     object
num_partes_afectadas                 int64
dtype: object

In [12]:
data.head(2)

,numero_siniestro,valor_vehiculo,descripcion_siniestro,1.- Sección delantera,2.- Lateral delantero izquierdo,3.- Lateral delantero derecho,4.- Lateral trasero izquierdo,5.- Lateral trasero derecho,6.- Sección posterior,7.- Techo,codigo_marca,codigo_clase,codigo_tipo_vehiculo,anio_modelo,municipio,label,valor_deducible,num_partes_afectadas
0,899767,68900000.0,"El día primero (1) de enero de 2022, siendo ap...",1,1,1,1,1,1,1,16,12,138,2012,GRANADA,0,10 con un minimo de 6 smmlv,7
1,899768,73700000.0,El vehículo venía de Istmina Chocó a recogerm...,1,1,1,1,1,1,1,30,6,134,2020,PEREIRA,1,950.000,7


In [15]:
data.to_csv("data_totallossco_training.csv", index = False)

In [16]:
###########################

Now we will use the optuner library to optimize the hyperparameters

You may need to download SQLAlchemy < 2.0 in order to Optuna run

I suggest to run the process for 200 trials in rounds of 40 (opt.optune(40)). It can take 2 days in a 
c5.4xlarge EC2 machine

In [17]:
#pip install nppm_latam_optuner

In [13]:
from nppm_latam_optuner.optuner import Optuner

/opt/cortex-installs/miniconda/envs/mlmodels/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


we define our predictors and label

In [14]:
X = data.drop(columns=['numero_siniestro', 'label'])

y = data.label
y.value_counts()

0    30693
1     2316
Name: label, dtype: int64

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42)

skf = StratifiedKFold(n_splits=4, random_state=0, shuffle=True)

opt = Optuner(study_name='co_totalloss', prediction_task='clf',
              X=X_train, y=y_train,
              num_cols=['valor_vehiculo', 'num_partes_afectadas', '1.- Sección delantera', '2.- Lateral delantero izquierdo',
               '3.- Lateral delantero derecho', '4.- Lateral trasero izquierdo','5.- Lateral trasero derecho', '6.- Sección posterior', '7.- Techo'],
              cat_cols=['codigo_marca', 'codigo_clase', 'codigo_tipo_vehiculo', 'anio_modelo', 'municipio', 'valor_deducible'],
              text_cols=['descripcion_siniestro'],
              language='spa',
              cv=skf, scoring='average_precision', direction='maximize',
              model_n_jobs=9, cross_val_n_jobs=4)

opt.optune(1)

Now we will infer in the test set

In [16]:
import joblib

optuned_model = joblib.load('co_totalloss.joblib')
type(optuned_model)

/opt/cortex-installs/miniconda/envs/mlmodels/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [14:08:25] WARNING: /workspace/src/common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


nppm_latam_optuner.optuner.OptunedModel

In [17]:
from sklearn.metrics import average_precision_score

probas = optuned_model.predict_proba(X_test)

X_test.shape, probas.shape

((6602, 16), (6602, 2))

In [18]:
probas

array([[0.7700663 , 0.22993365],
       [0.9431783 , 0.05682167],
       [0.99530894, 0.00469103],
       ...,
       [0.9599323 , 0.04006765],
       [0.9832779 , 0.01672207],
       [0.97026134, 0.02973862]], dtype=float32)

In [19]:
average_precision_score(y_test, probas[:, 1])

0.5230555567082098

In [ ]:
#######################